## Läs in filen, skriv ut den och städa den

In [1]:
import pandas as pd

data = pd.read_excel("Återbet.xlsx")

data = data[pd.to_numeric(data['OrderNo'], errors='coerce').notnull()]

data["OrderNo"] = data["OrderNo"].astype(int).replace('\.o', '', regex = True)

data["PriceIncVat"] = data["PriceIncVat"].astype(int).replace('\.o', '', regex = True)

data = data[["ProductName", "OrderNo", "NoOfItems", "PriceIncVat", "PriceExcVat", "Info"]]

data.dropna(subset=['ProductName'], inplace = True)


## Kontering Klarna

In [2]:
data2 = data.copy()

data2["Belopp"] = None

data2["PriceIncVat"] = data2["PriceIncVat"]*-1

data2["Konto"] = "1570"

data2["B"] = 0
data2["C"]= None
data2["D"] = None
data2["E"] = None

data2.loc[data2['Info'].str.contains('Kort'), 'Konto'] = "1571"

## Kontering frakt

In [3]:
# Skapa ett dataset med bara de som har "Hemleverans"
# samt att man bara får en rad av varje Ordernummer

t = data2[data2.Info.str.contains('Hemleverans')].drop_duplicates(subset='OrderNo')

# Använda lambda-funktionen för att ge orderbeloppen en fraktavgift på 19 kr

t = t['PriceIncVat'].apply(lambda x: x-19)

# Skapa en funktion för konteringen av frakten

def frakt(data):
    try:
        antal = t.count()
        split = [
        {'Konto':3507, 'B': 920, 'Belopp': 15.2*antal, 'Referens': 'Aterbet'},
        {'Konto':2610, 'B': 0, 'Belopp': 3.8*antal, 'Referens': 'Aterbet'},
        ] 
        split = pd.DataFrame(split)
        split["C"]= None
        split["D"] = None
        split["E"] = None
    except:
        pass
    return split[["Konto", "B", "C", "D", "E", "Belopp", "Referens"]]

data6 = frakt(data)

# Skapa en lista av alla index-nummer på de som ska ha fraktavgift

my_list= t.index.values.tolist()

# Ersätt index-listans tidigare belopp med det nya beloppet som inkluderar frakt

data2.at[my_list,"PriceIncVat"]= t

## Sätt upp strukturen för Klarna-dataframen

In [4]:
data2 = data2[["Konto", "B", "C", "D", "E", "PriceIncVat", "OrderNo"]]

data2.columns = ["Konto", "B", "C", "D", "E","Belopp", "Referens"]

## Kontering produkter

In [5]:
data["Konto"] = None

data["Referens"] = None

data["Konto"] = "2976"

data["Moms"] = data["PriceIncVat"]-data["PriceExcVat"]

def gruppaket_split(data):

    try:
        antal = data.NoOfItems[data.ProductName == 'Stora Gruppaketet'].sum()
        split = [
        {'Konto':2976, 'Belopp': 380*antal, 'Referens': '3054/110'},
        {'Konto':2976, 'Belopp': -96*antal, 'Referens': '3069/110'},
        {'Konto':2979, 'Belopp': 80*antal, 'Referens': '3923/110'},
        {'Konto':2610, 'Belopp': 71*antal, 'Referens': 'Aterbet'},
        ] 
        split = pd.DataFrame(split)
        split["B"] = 0
        split["C"]= None
        split["D"] = None
        split["E"] = None
    except:
        pass
    
    return split[["Konto", "B", "C", "D", "E", "Belopp", "Referens"]]

def guldkort_ÅA_split(data):
    try:
        antal = data.NoOfItems[data.ProductName == 'Guldkortet ÅA'].sum()
        split = [
        {'Konto': 2976, 'Belopp': 1359.2*antal, 'Referens': '3056/110'},
        {'Konto': 2976, 'Belopp': -80*antal, 'Referens': '3069/110'},
        {'Konto': 2610, 'Belopp': 319.8*antal, 'Referens': 'Aterbet'}
        ]
        split = pd.DataFrame(split)
        split["B"] = 0
        split["C"]= None
        split["D"] = None
        split["E"] = None
    except:
        pass
    
    return split[["Konto", "B", "C", "D", "E", "Belopp", "Referens"]]

data3 = gruppaket_split(data)

data4 = guldkort_ÅA_split(data)

data.loc[data['ProductName'].str.contains('Guldkortet'), 'Referens'] = '3056/110'
data.loc[data['ProductName'].str.contains('Gröna Kortet'), 'Referens'] = '3018/110'
data.loc[data['ProductName'].str.contains('Gröna kortet'), 'Referens'] = '3018/110'
data.loc[data['ProductName'].str.contains('Åkband'), 'Referens'] = '3051/110'
data.loc[data['ProductName'].str.contains('Paketentré'), 'Referens'] = '3001/110'
data.loc[data['ProductName'].str.contains('Entré'), 'Referens'] = '3001/110'
data.loc[data['ProductName'].str.contains('House'), 'Referens'] = '3059/110'
data.loc[data['ProductName'].str.contains('Rab'), 'Referens'] = '3011/110'
data.loc[data['ProductName'].str.contains('Stora'), 'Referens'] = 'Stora Gruppaketet'
data.loc[data['ProductName'].str.contains('Guldkortet ÅA'), 'Referens'] = 'ÅA'
data.loc[data['ProductName'].str.contains('Present'), 'Referens'] = "Aterbet"
data.loc[data['Info'].str.contains('Hämta'), 'Referens'] = "Aterbet"


data.loc[data['ProductName'].str.contains('House'), 'Konto'] = "2978"
data.loc[data['ProductName'].str.contains('Present'), 'Konto'] = "2894"
data.loc[data['ProductName'].str.contains('Stora'), 'Konto'] = "1915"
data.loc[data['ProductName'].str.contains('Guldkortet ÅA'), 'Konto'] = "1915"
data.loc[data['Info'].str.contains('Hämta'), 'Konto'] = "2813"

data["2610"] = None
data["2630"] = None

data["Belopp"] = None

data.loc[data['Konto'].str.contains('2976'), '2610'] = data["Moms"]
data.loc[data['Konto'].str.contains('2978'), '2630'] = data["Moms"]
data.loc[data['Konto'].str.contains('2610'), 'Referens'] = 'Aterbet'
data.loc[data['Konto'].str.contains('2630'), 'Referens'] = 'Aterbet'

data.loc[data['Konto'].str.contains('2976'), 'Belopp'] = data["PriceExcVat"]
data.loc[data['Konto'].str.contains('2978'), 'Belopp'] = data["PriceExcVat"]
data.loc[data['Konto'].str.contains('2813'), 'Belopp'] = data["PriceIncVat"]
data.loc[data['Konto'].str.contains('1915'), 'Belopp'] = data["PriceIncVat"]
data.loc[data['Konto'].str.contains('2894'), 'Belopp'] = data["PriceIncVat"]

Moms_25 = data["2610"].sum()
Moms_6 = data["2630"].sum()

data["B"] = 0
data["C"]= None
data["D"] = None
data["E"] = None

data = data[["Konto", "B", "C", "D", "E", "Belopp", "Referens"]]

data.loc[-1] = ["2610", 0, None, None, None, Moms_25, "Aterbet"]
data.loc[-2] = ["2630", 0, None, None, None, Moms_6, 'Aterbet']

## Sammanställ de olika dataframen, städa, och gör en csv-fil av dem

In [6]:
frames = [data, data2, data3, data4, data6]

data5 = pd.concat(frames)

data5 = data5[data5.Referens != 'Stora Gruppaketet']

data5 = data5[data5.Referens != 'ÅA']

data5 = data5[data5.Belopp != 0]

data5["Belopp"] = data5["Belopp"].astype(float).replace('\.o', '', regex = True)

data5.B = data5.B.astype(int)

data5['Referens'] = data5['Referens'].apply(lambda x: 'Aterbat' if x is None else x)

data7 = data5.groupby(by=['Referens','Konto','B']).sum().reset_index()

data7["C"]= None
data7["D"] = None
data7["E"] = None

data7 = data7[["Konto", 'B', 'C', 'D', 'E','Belopp', "Referens"]]

data7.to_csv('Återbetalning.csv', index=False, header=False, sep = ";") 